# Preliminary Notebook

## 1. Import and Install Dependencies

In [12]:
%pip install tensorflow-macos opencv-python mediapipe-silicon sklearn matplotlib
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [13]:
import cv2 # opencv
import numpy as np
import os # easier file path handling
from matplotlib import pyplot as plt # im.show for easy visualization
import time # to insert "sleep" in between frames
import mediapipe as mp # for accessing and reading from webcam

## 6. Preprocess Data and Create Labels and Features

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [15]:
# path for exported data (numpy arrays)
DATA_PATH = os.path.join('MP_Data_Jin-Ho')

# actions to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# 30 videos of data
no_sequences = 30

# each video with 30 frames
sequence_length = 30

In [16]:
# create label map (dict, our .json file)
label_map = {label:num for num, label in enumerate(actions)}

In [38]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [17]:
sequences, labels = [], [] # sequences will be x data, labels will be y data
# loop over all actions (words)
for action in actions: 
    # loop over all sequences (videos)
    for sequence in range(no_sequences): 
        window = [] # represents all frames of particular sequence (video)
        # loop through each frame
        for frame_num in range(sequence_length): 
            # load up current frame (frame_num)
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res) # append to one video
        sequences.append(window) # append all videos to sequence (for a word)
        labels.append(label_map[action])

In [18]:
X = np.array(sequences) # shape: (90, 30, 1662)

In [19]:
y = to_categorical(labels).astype(int) # one-hot encoded labels (words)
# shape: (90, 3)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## 7. Build and Train LSTM Neural Network

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [22]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# categorical_crossentropy must be used for multiclass classification model! 

In [25]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
# advantage of using mediapipe holistic model is you don't need additional data generator to build up a pipeline of data. Training data fits all into memory.

Epoch 1/2000


2023-04-05 15:18:45.012090: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 2s 39ms/step - loss: 1.2390 - categorical_accuracy: 0.2824
Epoch 2/2000
3/3 [==============================] - 0s 39ms/step - loss: 1.4621 - categorical_accuracy: 0.2706
Epoch 3/2000
3/3 [==============================] - 0s 39ms/step - loss: 1.1457 - categorical_accuracy: 0.3882
Epoch 4/2000
3/3 [==============================] - 0s 42ms/step - loss: 1.3925 - categorical_accuracy: 0.3412
Epoch 5/2000
3/3 [==============================] - 0s 38ms/step - loss: 1.1129 - categorical_accuracy: 0.3647
Epoch 6/2000
3/3 [==============================] - 0s 43ms/step - loss: 1.1025 - categorical_accuracy: 0.3647
Epoch 7/2000
3/3 [==============================] - 0s 43ms/step - loss: 1.1288 - categorical_accuracy: 0.2824
Epoch 8/2000
3/3 [==============================] - 0s 37ms/step - loss: 1.0929 - categorical_accuracy: 0.3529
Epoch 9/2000
3/3 [==============================] - 0s 35ms/step - loss: 1.1002 - categorical_accuracy: 0.3765
Epoch 10/2000


In [35]:
%load_ext tensorboard

In [35]:
# to run it in the notebook: 
%tensorboard --logdir=./Logs/train --port=8008

# to run it in terminal: python3 -m tensorboard --logdir=./Logs/train --port=8008
# then copy+paste this into your internet browser: localhost:8008

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

## Make Predictions

In [29]:
res = model.predict(X_test)

1/1 [==============================] - 1s 513ms/step


In [30]:
actions[np.argmax(res[0])] # predictions

'hello'

In [31]:
actions[np.argmax(y_test[0])] # actual

'hello'

## 9. Save Weights

In [32]:
model.save('action.h5')

In [33]:
del model

In [34]:
model.load_weights('action.h5')

NameError: name 'model' is not defined

## 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
# convert one-hot encoded categories to labels, e.g. 0, 1 and 2
# instead of [1,0,0], [0,1,0], [0,0,1]
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = 